In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geoviews
import holoviews as hv
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [ ]:
# set the city variables
target_city = "Melbourne, Australia"
city = "Melbourne"

# Build the latitude search URL
url = f"https://api.geoapify.com/v1/geocode/search?text={target_city}&format=json&apiKey={geoapify_key}"

# Run a request to endpoint and convert result to json
results = requests.get(url).json()

#create the co-ordinate variables
lat = results["results"][0]["lat"]
lon = results["results"][0]["lon"]

# print the results
print(target_city, lat, lon)

In [ ]:
# set search variable
cities = ["Melbourne"]

# creates lists to hold the search results
school_name = []
school_address = []
school_postcode =[]
school_lat = []
school_lon = []
school_distance = []
school_type = []
school_grades = []

# set the search parameters
radius = 15000
categories = "education.school"

# set the parameters for the type of search
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"
limit = 100

# set the search URL
base_url = "https://api.geoapify.com/v2/places"

# create the parameters dictionairy
params = {"categories": categories, "apiKey": geoapify_key, "limit": limit, "filter": filters, "bias": bias}

# create loop to to search for schools
for city in cities:
    response = requests.get(base_url, params=params).json()

# set the variable to bring back multiple results
    num_features = len(response["features"])

# create the loop to append results to each list
    for i in range(min(num_features,100)):
        school_name.append(response["features"][i]["properties"]["address_line1"])
        school_address.append(response["features"][i]["properties"]["address_line2"])
        school_postcode.append(response["features"][i]["properties"]["postcode"])
        school_lat.append(response["features"][i]["geometry"]["coordinates"][1])
        school_lon.append(response["features"][i]["geometry"]["coordinates"][0])
        school_distance.append(response["features"][i]["properties"]["distance"])
        
# create the loop if data is not available for school type or grade
        if "operator:type" in response["features"][i]["properties"]["datasource"]["raw"]:
            school_type.append(response["features"][i]["properties"]["datasource"]["raw"]["operator:type"])
        else:
            school_type.append("N/A")
        if "grades" in response["features"][i]["properties"]["datasource"]["raw"]:
            school_grades.append(response["features"][i]["properties"]["datasource"]["raw"]["grades"])
        else:
            school_grades.append("N/A")

# take results in the lists and create a dataframe
school_results_df = pd.DataFrame({
    "school name": school_name,
    "address": school_address,
    "postcode": school_postcode,  
    "latitude": school_lat,
    "longitude": school_lon,
    "distance from CBD": school_distance,
    "school type": school_type,
    "school grades": school_grades,
})

# show results in dataframe
school_results_df.head()

In [ ]:
# remove schools that only accomodate to primary school children
school_results_df = school_results_df[(school_results_df["school grades"] != "0-6") & (school_results_df["school grades"] != "P-6") & (school_results_df["school grades"] != "N/A")]

#show the results
school_results_df.head()

In [ ]:
#reset the index
school_results_df = school_results_df.reset_index(drop=True)

#show the results
school_results_df.head()

In [ ]:
# convert school grades to a string
school_results_df["school grades"] = school_results_df["school grades"].astype(str)

# print the dtypes for the dataframe
print(school_results_df.dtypes)

In [ ]:
%%capture --no-display

# Configure the map plot for shopping centre data
school_plot = school_results_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    global_extent = False,
    tiles = "OSM",
    hover_cols = "all",
    frame_width = 700,
    frame_height = 500,
    size = 100,
    color = "school name",
)

# show the map plot
school_plot

In [ ]:
# save school map image
# school_plot.save("schoolmap.png")
# plt.savefig("schoolmap.png")

In [ ]:
# set the search variable
cities = ["Melbourne"]

# creates lists to hold the search results
mall_name = []
mall_address = []
mall_postcode =[]
mall_lat = []
mall_lon = []
mall_distance = []

# set the search parameters
radius = 15000
categories = "commercial.shopping_mall"

# set the parameters for the type of search
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"
limit = 50

# set the search URL
base_url = "https://api.geoapify.com/v2/places"

# create the parameters dictionairy
params = {"categories": categories, "apiKey": geoapify_key, "filter": filters, "limit": limit, "bias": bias}

# create loop to search for shopping centres
for city in cities:
    shopping_response = requests.get(base_url, params=params).json()
    num_features = len(shopping_response["features"])
    
# create the loop to append results to each list
    for i in range(min(num_features,50)):
        mall_name.append(shopping_response["features"][i]["properties"]["address_line1"])
        mall_address.append(shopping_response["features"][i]["properties"]["address_line2"])
        mall_postcode.append(shopping_response["features"][i]["properties"]["postcode"])
        mall_lat.append(shopping_response["features"][i]["geometry"]["coordinates"][1])
        mall_lon.append(shopping_response["features"][i]["geometry"]["coordinates"][0])
        mall_distance.append(shopping_response["features"][i]["properties"]["distance"])

# take results in the lists and create a dataframe
shopping_results_df = pd.DataFrame({
    "shopping centre": mall_name,
    "address": mall_address,
    "postcode": mall_postcode,  
    "latitude": mall_lat,
    "longitude": mall_lon,
    "distance from CBD": mall_distance,
})

# show results
shopping_results_df.head()

In [ ]:
%%capture --no-display

# Configure the map plot for shopping centre data
shopping_plot = shopping_results_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    global_extent = False,
    tiles = "OSM",
    hover_cols = "all",
    frame_width = 700,
    frame_height = 500,
    size = 100,
    color = "shopping centre",
)

# show the map plot
shopping_plot

In [ ]:
# Save the map
# shopping_plot.save("shoppingmap.png")
# plt.savefig("shoppingmap.png")

In [ ]:
# set search variable
cities = ["Melbourne"]

# creates lists to hold the required information results
station_name = []
station_address = []
station_postcode =[]
station_lat = []
station_lon = []
station_distance = []

# set the search parameters
radius = 15000
categories = "public_transport.train"

# set the parameters for the type of search
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"
limit = 200

# set the search URL
base_url = "https://api.geoapify.com/v2/places"

# create the parameters dictionairy
params = {"categories": categories, "apiKey": geoapify_key, "filter": filters, "limit": limit, "bias": bias}

# create loop to search for train stations
for city in cities:
    response = requests.get(base_url, params=params).json()
    num_features = len(response["features"])
    
# create the loop to append results to each list
    for i in range(min(num_features,200)):
        station_name.append(response["features"][i]["properties"]["address_line1"])
        station_address.append(response["features"][i]["properties"]["address_line2"])
        station_postcode.append(response["features"][i]["properties"]["postcode"])
        station_lat.append(response["features"][i]["geometry"]["coordinates"][1])
        station_lon.append(response["features"][i]["geometry"]["coordinates"][0])
        station_distance.append(response["features"][i]["properties"]["distance"])

# take results in the lists and create a dataframe
train_results_df = pd.DataFrame({
    "station name": station_name,
    "address": station_address,
    "postcode": station_postcode,  
    "latitude": station_lat,
    "longitude":station_lon,
    "distance from CBD": station_distance,
})

# show results
train_results_df

In [ ]:
%%capture --no-display

# Configure the map plot for train station data
transport_plot = train_results_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    global_extent = False,
    tiles = "OSM",
    hover_cols = "all",
    frame_width = 700,
    frame_height = 500,
    size = 50,
    color = "station name",
)

# show the map plot
transport_plot

In [ ]:
# Save the map
# transport_plot.save("transportmap.png")
# plt.savefig("transportmap.png")